In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import librosa
from torch.utils import data
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from scipy.misc import imresize

In [17]:
path = "/home/ddmnn3/PycharmProjects/ADOS/Data/clip_images/all/"
EPOCHS = 10
BATCH_SIZE = 32
learning_rate = 1e-4
counter = 0
LOG_INTERVAL = 10
no_of_sample = 20000
ZDIMS = 16


In [18]:
class ADOSClipImg(Dataset):

    def __init__(self, root_dir, im_name_list, resize_dim, transform=None):
        self.root_dir = root_dir
        self.im_list = im_name_list
        self.resize_dim = resize_dim
        self.transform = transform

    def __len__(self):
        return len(self.im_list)

    def __getitem__(self, idx):
        im = Image.open(os.path.join(self.root_dir, self.im_list[idx]))
        im = np.array(im)
        im = im[...,:3]
        im = imresize(im, self.resize_dim, interp='nearest')
        im = im / 255

        if self.transform:
            im = self.transform(im)

        return im
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors. numpy image: H x W x C, torch image: C X H X W
    """

    def __call__(self, image, invert_arrays=True):

        if invert_arrays:
            image = image.transpose((2, 0, 1))

        return torch.from_numpy(image) 

In [27]:
ZDIMS

16

In [ ]:
class Conv_Block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, stride, pool_kernel_size=(2, 2)):
        super(Conv_Block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, padding, stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, padding, stride)
        self.pool = nn.MaxPool2d(pool_kernel_size)

    def forward(self, x):
        x = F.elu(self.conv1(x))
        x = F.elu(self.conv2(x))
        x = self.pool(x)

        return x

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        # Encoder
        self.conv1 = nn.Conv2d(3, 256, (4, 4), 1, 2)
        self.norm1 = nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(256, 128, (4, 4), 1, 2)
        self.norm1 = nn.BatchNorm2d(128)
        self.conv2 = nn.Conv2d(128, 64, (4, 4), 1, 2)
        self.norm1 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(8192, 928)
        self.fc2 = nn.Linear(928, 512)
        self.fc31  = nn.Linear(512, ZDIMS)  
        self.fc32  = nn.Linear(512, ZDIMS)

        self.fc4 = nn.Linear(ZDIMS, 512)
        self.fc5 = nn.Linear(512, 928)
        self.fc6 = nn.Linear(928, 928)


        self.deconv1 = nn.ConvTranspose2d(32, 64, (1,1), 1)
        self.norm4 = nn.BatchNorm2d(64)
        self.deconv2 = nn.ConvTranspose2d(64, 128, (1,1), 1)
        self.norm5 = nn.BatchNorm2d(128)
        self.deconv3 = nn.ConvTranspose2d(128, 256, (1,1), 1)
        self.norm6 = nn.BatchNorm2d(256)
                
    def encode(self, x):
        c1 = F.leaky_relu(self.norm1(self.conv1(x)))
        c2 = F.leaky_relu(self.norm2(self.conv2(c1)))
        c3 = F.leaky_relu(self.norm3(self.conv3(c2)))
        ev = c3.view(c3.size(0), -1)
        h1 = F.relu(self.fc1(ev))
        h2 = F.relu(self.fc2(h1))
        return F.relu(self.fc31(h2)), F.relu(self.fc32(h2))
    

    def reparameterize(self, mean, var):
        std = var.mul(0.5).exp_()
        esp = torch.randn(*mean.size())
        std = std.to(device)
        esp = esp.to(device)
        z = mean + std * esp
        return z

    def decode(self, z):
        h3 = F.relu(self.fc4(z))
        h4 = F.relu(self.fc5(h3))
        h5 = F.relu(self.fc6(h4))
        dv = h5.view(-1, 32, 128, 1)  
        dc1 = F.leaky_relu(self.norm4(self.deconv1(dv)))
        dc2 = F.leaky_relu(self.norm5(self.deconv2(dc1)))
        return F.sigmoid(self.norm6(self.deconv3(dc2)))


    def forward(self, x):
        mean, var =  self.encode(x)
        z = self.reparameterize(mean, var)
        return self.decode(z), mean, var, z



In [22]:
def lossfn(x, target, mean, logvar):
    bce = nn.BCELoss()
    bce_loss = bce(x, target)
    
    scaling_factor = out.shape[0]*out.shape[1]*out.shape[2]*out.shape[3]
    
    kl_loss = -.5 * torch.sum(1 + logvar - mean**2 - torch.exp(logvar))
    kl_loss /= scaling_factor
    
    return bce_loss + kl_loss

In [ ]:
model = VAE().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [25]:
data = Variable(data.type(torch.FloatTensor))
data = data.cuda()

In [26]:
recon_batch, mu, logvar, _ = model(data)

RuntimeError: running_mean should contain 256 elements not 64

In [ ]:
for epoch in range(num_epochs):
    print("Epoch #" + str(epoch))
    for idx, (spec) in enumerate(dataSet):
        optimizer.zero_grad()
        out, mean, logvar, _ = model(spec.cuda())
        if out.shape[0] == batch_size:
            out = out.reshape(batch_size,1,257,29)
        else:
            out = out.reshape(out.shape[0],1,257,29)
        loss = lossfn(out, spec.cuda(), mean, logvar)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        torch.cuda.empty_cache()
    if loss > 0.0:
            if loss < low_loss:
                print('saving model')
                torch.save(model.state_dict(), "/home/michael/Downloads/model/" + "model1")
                low_loss = loss
                print(loss)

In [7]:
def train(epoch):
    # toggle model to train mode
    model.train()
    #train_loss = 0
    # in the case of MNIST, len(train_loader.dataset) is 60000
    # each `data` is of BATCH_SIZE samples and has shape [128, 1, 28, 28]
    for batch_idx, data in enumerate(train_loader):
        data = Variable(data.type(torch.FloatTensor))
        data = data.cuda()
        optimizer.zero_grad()

        # push whole batch of data through VAE.forward() to get recon_loss
        recon_batch, mu, logvar, _ = model(data)
        # calculate scalar loss
        loss = model.loss_function(recon_batch, data, mu, logvar)
        # calculate the gradient of the loss w.r.t. the graph leaves
        # i.e. input variables -- by the power of pytorch!
        loss.backward()
        #train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [8]:
image_files = os.listdir(path)
train_dataset = ADOSClipImg(path, image_files, (128, 128), transforms.Compose([ToTensor()]))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)

#Take only 1000 images in test
#test_dataset = CelebaDataset(root_dir, image_files[100000:101000], (128, 128), transforms.Compose([ToTensor()]))
#test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=10, shuffle=True)

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    torch.save(model.state_dict(), "/home/ddmnn3/PycharmProjects/ADOS/Models/" + "model")

/home/ddmnn3/anaconda3/envs/spectre2/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  app.launch_new_instance()
/home/ddmnn3/anaconda3/envs/spectre2/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  app.launch_new_instance()
/home/ddmnn3/anaconda3/envs/spectre2/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  app.launch_new_instance()
/home/ddmnn3/anaconda3/envs/spectre2/lib/python3.6/site-packages/ipykernel_launcher.py:1

RuntimeError: shape '[-1, 100352]' is invalid for input of size 11505664

In [11]:
image_files = []
for file in os.listdir(path):
    image_files.append(file)


In [39]:
import random
select = [random.randint(0, 20000) for p in range(0, 300)]

In [46]:
test_images = image_files[1:300]

In [48]:
test_dataset = ADOSClipImg(path, test_images, (256, 256), transforms.Compose([ToTensor()]))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)


In [13]:
im = Image.open(path+image_files[1])
im = np.array(im)
im = im[...,:3]
im = imresize(im, (128,128), interp='nearest')
im = im / 255

/home/ddmnn3/anaconda3/envs/spectre2/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  after removing the cwd from sys.path.


In [14]:
im.shape

(128, 128, 3)

In [59]:
model(torch.cuda.FloatTensor(im))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 4, 3, 3], but got 3-dimensional input of size [256, 256, 4] instead

In [ ]:
batch_idx, data = train_loader[0]
data = Variable(data.type(torch.FloatTensor))
data = data.cuda()

recon_batch, mu, logvar = model(data)

In [11]:
model.eval()

VAE(
  (block1): Conv_Block(
    (conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Conv_Block(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Conv_Block(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (block4): Conv_Block(
    (conv1): Conv2d(256, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_s

In [ ]:
z_test = model.reparameterize(model.encode(x))


In [ ]:
def lossfn(x, target, mean, logvar):
    bce = nn.BCELoss()
    bce_loss = bce(x, target)

    scaling_factor = out.shape[0] * out.shape[1] * out.shape[2] * out.shape[3]

    kl_loss = -.5 * torch.sum(1 + logvar - mean ** 2 - torch.exp(logvar))
    kl_loss /= scaling_factor

    return bce_loss + kl_loss

In [ ]:
model.train()
train_loss = 0

In [ ]:
data.shape

In [ ]:


for epoch in range(2):
    print("Epoch #" + str(epoch))
    
    for batch_idx, data in enumerate(train_loader):
        data = Variable(data.type(torch.FloatTensor))
        data = data.cuda()
        optimizer.zero_grad()
        out, mean, logvar, _ = model(data)
        loss = lossfn(out, data, mean, logvar)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.data[0] / len(data)))
            torch.save(model.state_dict(), "/home/ddmnn3/PycharmProjects/ADOS/Models/" + "model1")
            
        optimizer.zero_grad()
        torch.cuda.empty_cache()

In [ ]:
for epoch in range(1, EPOCHS + 1):
        train(epoch, model, optimizer, train_loader)
        #test(epoch, model, test_loader)

        

In [ ]:
# 64 sets of random ZDIMS-float vectors, i.e. 64 locations / MNIST
# digits in latent space
sample = Variable(torch.randn(64, 16, 8, 8))
if CUDA:
    sample = sample.cuda()
sample = model.decode(sample).cpu()

# save out as an 8x8 matrix of MNIST digits
# this will give you a visual idea of how well latent space can generate things
# that look like digits
save_image(sample.data.view(64, 3, 128, 128), './celeba/reconstruction' + str(epoch) + '.png')